<a href="https://colab.research.google.com/github/kgpark88/deeplearning/blob/master/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ABC 데이터셋을 활용하여 이미지 분류기를 만들고 검증손실이 최소인 모델을 파일로 저장하세요.
## [조건]  
- 이미지 높이: 224픽셀
- 이미지 길이: 224픽셀
- 색상 채널: 3
- 모델파일명: abc.model

## 라이브러리 임포트(import)

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16


## 학습데이터와 검증데이터셋 준비
## [데이터셋 종류]
- 사람 표정 이미지
- 제품 양품 불량 이미지

In [2]:
dataset_name = 'cats_vs_dogs'
train_dataset = tfds.load(name=dataset_name, split='train[:80%]')
valid_dataset = tfds.load(name=dataset_name, split='train[80%:]')

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteWZF2SO/cats_vs_dogs-train.tfrecord


Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


# 이미지 변환 함수 작성

In [3]:
def preprocess(data):
    x = data['image']
    y = data['label']
    x = x / 255
    x = tf.image.resize(x, size=(224, 224))
    return x, y

## 모델 구성/빌드(build)/저장

In [ ]:
batch_size = 32
train_data = train_dataset.map(preprocess).batch(batch_size)
valid_data = valid_dataset.map(preprocess).batch(batch_size)

transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
transfer_model.trainable = False

model = Sequential([
    transfer_model,
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    # YOUR CODE HERE, BUT MAKE SURE YOUR LAST LAYER HAS 2 NEURONS ACTIVATED BY SOFTMAX
    tf.keras.layers.Dense(2, activation='softmax')
])

checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(train_data,
          validation_data=(valid_data),
          epochs=20,
          callbacks=[checkpoint],
          )

58892288/58889256 [==============================] - 0s 0us/step
Epoch 1/20
582/582 [==============================] - 102s 161ms/step - loss: 0.5151 - acc: 0.8363 - val_loss: 0.2018 - val_acc: 0.9123

Epoch 00001: val_loss improved from inf to 0.20183, saving model to my_checkpoint.ckpt
Epoch 2/20
236/582 [===========>..................] - ETA: 44s - loss: 0.1974 - acc: 0.9202

In [ ]:
np.round(model.predict(X_new), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

## 모델 서빙용 모델 저장

In [ ]:
model_version = "001"
model_name = "abc_model"
model_path = os.path.join(model_name, model_version)
model_path

'abc_model/001'

In [ ]:
!rm -rf {model_name}

In [ ]:
tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: abc_model/001/assets


In [ ]:
for root, dirs, files in os.walk(model_name):
    indent = '    ' * root.count(os.sep)
    print('{}{}/'.format(indent, os.path.basename(root)))
    for filename in files:
        print('{}{}'.format(indent + '    ', filename))

abc_model/
    001/
        saved_model.pb
        assets/
        variables/
            variables.data-00000-of-00001
            variables.index


## 저장된 모델 확인

In [ ]:
!saved_model_cli show --dir {model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['flatten_input'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 28, 28, 1)
        name: serving_default_flatten_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict
W0204 02:14:10.220359 139938635921280 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/p

## 모델 서빙 서비스 구현
### 아래 방법 중 선택
- 자체 구현(Flask, Django)
- TensorFlow Serving
- TorchServ 

예시1) TensorFlow Serving [Docker](https://docs.docker.com/install/)를 설치하고 실행하세요. REST API 서비스 포트는 8888번으로 지정하세요.

```bash
docker pull tensorflow/serving

export ML_PATH=$HOME/ml # or wherever this project is
docker run -it --rm -p 7777:7777 -p 8888:8888 \
   -v "$ML_PATH/abc_model:/models/abc_model" \
   -e MODEL_NAME=abc_model \
   tensorflow/serving
```

예시2) tensorflow_model_server 를 설치하고 실행하세요. REST API 서비스 포트는 8888번으로 지정하세요.

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" > /etc/apt/sources.list.d/tensorflow-serving.list
!curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update && apt-get install -y tensorflow-model-server
!pip install -q -U tensorflow-serving-api

In [ ]:
os.environ["MODEL_DIR"] = os.path.split(os.path.abspath(model_path))[0]

In [ ]:
os.environ["MODEL_DIR"]

'/content/abc_model'

In [ ]:
%%bash --bg
nohup tensorflow_model_server \
     --rest_api_port=8888 \
     --model_name=abc_model \
     --model_base_path="${MODEL_DIR}" >server.log 2>&1

Starting job # 2 in a separate thread.


In [ ]:
!tail server.log

2021-02-04 03:04:16.691636: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:190] Running initialization op on SavedModel bundle at path: /content/abc_model/001
2021-02-04 03:04:16.694218: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:277] SavedModel load for tags { serve }; Status: success: OK. Took 32197 microseconds.
2021-02-04 03:04:16.694640: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at /content/abc_model/001/assets.extra/tf_serving_warmup_requests
2021-02-04 03:04:16.694742: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: abc_model version: 1}
2021-02-04 03:04:16.695637: I tensorflow_serving/model_servers/server.cc:371] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] getaddrinfo: address family for nodename not supported
[evhttp_server.cc : 223] NET_LOG: Couldn't bind to port 8888
[evhttp_server.cc : 63] NET_LOG: Server has not been terminated. For

## REST API로 서비스를 요청(request)하고 응답(resposne) 결과를 출력하세요.
- 요청 데이터 양식 : JSON
```
{
    "signature_name": "serving_default",
    "instances": data,
}
```


In [ ]:
import json

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist(),
})

In [ ]:
import requests

SERVER_URL = 'http://localhost:8888/v1/models/abc_model:predict'
response = requests.post(SERVER_URL, data=input_data_json)
response.raise_for_status() # raise an exception in case of error
response = response.json()

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.97, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])